In [41]:
import tensorflow as tf
import numpy as np
from json import load
from spacy.util import minibatch, compounding
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.preprocessing import normalize
import re
from spacy.training.example import Example
nlp = spacy.load("ru_core_news_sm")


In [42]:
with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)
ner = nlp.get_pipe("ner")
punctuation = list(map(lambda a: a, punctuation)) + \
    ['(', ')', '"', '«', '»', "'"]


In [43]:
def to_describer(text, ent):
    return (text, {"entities": ent})


In [45]:
TRAIN_DATA = []
for events in discription_of_event:
    description = discription_of_event[events]["описание"]
    new_skills = discription_of_event[events]["new_skills"]
    for skill in new_skills:
        TRAIN_DATA.append(to_describer(description, [(position.start(), position.end(), "type") for position in re.finditer(skill, description)]))
        
len(TRAIN_DATA)


39

In [46]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [
    pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# shuffle(TRAIN_DATA)

losses = {}
for batch in spacy.util.minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, drop=0.3)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "TON
Svet – это
образовательно-консультационное соо..." with entities "[(514, 567, 'type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
